<img width="50" src="https://carbonplan-assets.s3.amazonaws.com/monogram/dark-small.png" style="margin-left:0px;margin-top:20px"/>

# Figure 3: Future projections for each impact

_Author: Oriana Chegwidden_

The methods below conduct the analyses to calculate the fire risk change factors
included the manuscript <<< insert doi >>>.


In [ ]:
%load_ext autoreload
%autoreload 2

from carbonplan_forest_risks import load, setup, plot, fit, utils, prepare, collect
import xarray as xr
import numpy as np
from carbonplan_forest_risks.utils import get_store

In [ ]:
coarsen = 4

In [ ]:
mask = (
    (
        load.nlcd(store="az", year=2001).sel(band=[41, 42, 43, 90]).sum("band")
        > 0.25
    )
    .astype("float")
    .coarsen(x=coarsen, y=coarsen, boundary="trim")
    .mean()
)

# Load in fire data


In [ ]:
historical_fire = xr.open_zarr(
    get_store(
        "carbonplan-forests", "risks/results/paper/fire_terraclimate_v6.zarr"
    )
).load()
fire_mask = ~np.isnan(historical_fire.historical.isel(time=0).drop("time"))

In [ ]:
ds = (
    xr.open_zarr(
        get_store("carbonplan-forests", "risks/results/paper/fire_cmip.zarr")
    )
    .assign_coords({"x": mask.x, "y": mask.y})
    .where(fire_mask)
    .groupby("time.year")
    .sum()
    .where(fire_mask)
    .mean(dim=["x", "y"])
    .probability.mean(dim="gcm")
    .compute()
)

In [ ]:
for scenario in ["ssp245", "ssp370", "ssp585"]:
    ts = ds.sel(scenario=scenario)
    hist_slice = slice(1990, 2019)
    fut_slice = slice(2080, 2099)
    change = (
        ts.sel(year=fut_slice).mean(dim="year")
        / ts.sel(year=hist_slice).mean(dim="year").values
    )
    print("change factor for {} is {}".format(scenario, change.values))